In [1]:
import numpy as np
import torch
import random
seed=0
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)

# load data

In [2]:
from glob import glob
import os
images=[]
labels=[]
for folder in os.listdir('../dataset/wrist_xray'):
    for pic in glob('../dataset/wrist_xray/{}/*.jpg'.format(folder)):
        images.append(pic)
        labels.append(folder)
        

In [3]:
import pandas as pd
data=pd.DataFrame(zip(images,labels),columns=['img','label'])
data.label=data.label.map({'Fracture':0,'Normal':1})
data = data.sample(frac=1).reset_index(drop=True)
data.head()

,img,label
0,../dataset/wrist_xray/Normal\1.jpg,1
1,../dataset/wrist_xray/Normal\30.jpg,1
2,../dataset/wrist_xray/Fracture\47 Male (A View...,0
3,../dataset/wrist_xray/Normal\20.jpg,1
4,../dataset/wrist_xray/Normal\52.jpg,1


# top 5 lightest models in torchvision
1	squeezenet1_1	1235496  
2	shufflenet_v2_x0_5	1366792  
3	mnasnet0_5	2218512  
4	mobilenet_v2	3504872  
5	densenet121	7978856  `

In [4]:
import sys
sys.path.append("../src") 

In [5]:
from models.squeezenet import squeezenet1_1
from models.mymodels import mymodel1,mymodel3
from models.shufflenet import shufflenet_v2_x0_5
from models.mnastnet import mnasnet0_5
from models.densenet import densenet121
from models.mobilenet import mobilenet
from cross_vals import kfoldcv

In [6]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)/1e6

In [7]:
print('MyNet parameters :' ,count_parameters(mymodel1()))
print('SqueezeNet parameters :', count_parameters(squeezenet1_1()))
print('ShuffleNet parameters :', count_parameters(shufflenet_v2_x0_5()))
print('MnastNet parameters :', count_parameters(mnasnet0_5()))
print('MobileNet parameters :', count_parameters(mobilenet()))
print('DenseNet parameters :', count_parameters(densenet121()))


MyNet parameters : 1.158232
SqueezeNet parameters : 1.234344
ShuffleNet parameters : 1.36636
MnastNet parameters : 2.218224
MobileNet parameters : 3.504296
DenseNet parameters : 7.972584


In [8]:
epoch=32
batchsize=8
lr=0.001
skip_tuning=True
aug=0

dim='gray' for gray scale images  
skip_tuning=False, no grid search applied for hyper-parameter tuning,  
aug=1-> pytorch augmentation  
aug=0-> no augmentation  
aug=2-> albumentation augmentation    
aug=3-> rand augmentation

# OurNet

In [9]:
%%time
train_cv,clf_report=kfoldcv(model=mymodel3(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=skip_tuning,aug=aug)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.604   0.546     0.550
Class 1       0.448   0.512     0.452
macro         0.526   0.530     0.500
weighted      0.552   0.512     0.506
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.648   0.790     0.712
Class 1       0.590   0.414     0.484
macro         0.620   0.604     0.598
weighted      0.636   0.638     0.626
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.750   0.748     0.732
Class 1       0.668   0.650     0.632
macro         0.708   0.700     0.684
weighted      0.730   0.696     0.692
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.682   0.666     0.668
Class 1       0.562   0.580     0.562
macro         0.622   0.622     0.616
weighted      0.640   0.634     0.630
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.51,0.64,0.77,0.59
1,0.62,0.64,0.67,0.62
2,0.46,0.67,0.69,0.72
3,0.50,0.61,0.61,0.61
4,0.47,0.63,0.74,0.63
mean,0.512,0.638,0.696,0.634


# SqueezeNet

In [10]:
%%time
train_cv,clf_report=kfoldcv(model=squeezenet1_1(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=skip_tuning,aug=aug)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.670   0.576     0.614
Class 1       0.522   0.624     0.562
macro         0.596   0.600     0.588
weighted      0.620   0.596     0.596
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.596   0.912     0.712
Class 1       0.520   0.152     0.218
macro         0.558   0.534     0.466
weighted      0.588   0.584     0.512
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.650   0.644     0.642
Class 1       0.518   0.522     0.510
macro         0.584   0.584     0.576
weighted      0.608   0.588     0.594
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.676   0.556     0.578
Class 1       0.430   0.492     0.448
macro         0.552   0.524     0.516
weighted      0.580   0.556     0.544
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.69,0.64,0.67,0.67
1,0.54,0.56,0.56,0.49
2,0.67,0.59,0.56,0.54
3,0.58,0.58,0.47,0.53
4,0.50,0.55,0.68,0.55
mean,0.596,0.584,0.588,0.556


# ShuffleNet

In [11]:
%%time
train_cv,clf_report=kfoldcv(model=shufflenet_v2_x0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=skip_tuning,aug=aug)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.604   0.598     0.598
Class 1       0.462   0.474     0.460
macro         0.532   0.536     0.528
weighted      0.558   0.538     0.540
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.604   0.882     0.706
Class 1       0.324   0.212     0.248
macro         0.464   0.548     0.478
weighted      0.490   0.584     0.508
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.582   0.602     0.588
Class 1       0.430   0.420     0.416
macro         0.506   0.512     0.502
weighted      0.530   0.516     0.516
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.590   0.616     0.598
Class 1       0.444   0.416     0.424
macro         0.516   0.516     0.510
weighted      0.542   0.526     0.530
--------------------------Accuracy Tab

d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,DT,SVM,LR,KNN
0,0.64,0.56,0.54,0.56
1,0.62,0.59,0.51,0.56
2,0.59,0.69,0.64,0.54
3,0.47,0.53,0.47,0.50
4,0.37,0.55,0.42,0.47
mean,0.538,0.584,0.516,0.526


# Mnasnet

In [12]:
%%time
train_cv,clf_report=kfoldcv(model=mnasnet0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=skip_tuning,aug=aug)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.610   0.612     0.608
Class 1       0.478   0.466     0.468
macro         0.542   0.538     0.534
weighted      0.568   0.550     0.554
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.652   0.930     0.758
Class 1       0.634   0.302     0.368
macro         0.644   0.616     0.564
weighted      0.650   0.654     0.590
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.646   0.714     0.678
Class 1       0.550   0.472     0.508
macro         0.600   0.594     0.592
weighted      0.616   0.616     0.612
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.620   0.732     0.662
Class 1       0.536   0.386     0.420
macro         0.578   0.558     0.540
weighted      0.596   0.582     0.564
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.56,0.49,0.51,0.38
1,0.54,0.62,0.49,0.56
2,0.46,0.64,0.72,0.62
3,0.66,0.76,0.68,0.74
4,0.53,0.76,0.68,0.61
mean,0.55,0.654,0.616,0.582


# MobileNet

In [13]:
%%time
train_cv,clf_report=kfoldcv(model=mobilenet(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=skip_tuning,aug=aug)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.520   0.510     0.508
Class 1       0.378   0.376     0.368
macro         0.450   0.442     0.436
weighted      0.470   0.450     0.452
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.606   0.872     0.704
Class 1       0.526   0.238     0.282
macro         0.568   0.556     0.492
weighted      0.586   0.588     0.520
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.648   0.632     0.626
Class 1       0.496   0.522     0.488
macro         0.576   0.576     0.556
weighted      0.598   0.570     0.566
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.590   0.546     0.538
Class 1       0.408   0.494     0.408
macro         0.500   0.518     0.472
weighted      0.526   0.498     0.476
--------------------------Accuracy Tab

d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,DT,SVM,LR,KNN
0,0.38,0.56,0.59,0.46
1,0.41,0.56,0.64,0.62
2,0.41,0.72,0.59,0.54
3,0.50,0.55,0.53,0.55
4,0.55,0.55,0.50,0.32
mean,0.45,0.588,0.57,0.498


# DenseNet121

In [14]:
%%time
train_cv,clf_report=kfoldcv(model=densenet121(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=skip_tuning,aug=aug)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.674   0.668     0.668
Class 1       0.560   0.566     0.552
macro         0.616   0.616     0.610
weighted      0.638   0.622     0.622
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.694   0.780     0.726
Class 1       0.652   0.538     0.570
macro         0.674   0.658     0.648
weighted      0.690   0.670     0.660
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.760   0.748     0.744
Class 1       0.660   0.684     0.652
macro         0.710   0.718     0.700
weighted      0.734   0.704     0.708
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.614   0.590     0.592
Class 1       0.472   0.504     0.470
macro         0.544   0.546     0.532
weighted      0.570   0.544     0.544
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.56,0.64,0.69,0.49
1,0.72,0.69,0.59,0.38
2,0.59,0.67,0.74,0.62
3,0.66,0.74,0.82,0.68
4,0.58,0.61,0.68,0.55
mean,0.622,0.67,0.704,0.544
